# Лекция 8. QR разложение и способы его вычисления

## В прошлый раз

- PLU разложение
- Ряд Неймана
- Число обусловленности

## План на сегодня

- Что такое QR разложение?
- Всегда ли оно существует?
- Как вычислить?

## Общая концепция матричных разложений

Вычислительная линейная алгебра занимается решением следующих задач:

- Решение линейных систем $Ax = f$
- Вычисление собственных значений и векторов
- Вычисление сингулярных значений и векторов 
- Вычисление обратных матриц и иногда детерминантов
- Вычисление **матричных функций** таких как $\exp(A), \cos(A)$ (это не поэлементные функции!)

Для решения таких задач мы представляемм матрицу в виде суммы и/или произведения матриц **более простой структуры**, таких что мы можем решить эти задачи быстрее и/или более устойчивым образом.

Что такое **более простая структура**?

## Матрицы простой структуры

Мы уже упоминали некоторые классы структурированных матриц. 

Для плотных матриц это следующие матрицы

- **унитарные матрицы**
- **нижне- верхнетреугольные матрицы** 
- **диагональные матрицы**

## QR разложение

- Как следует из названия, это представление матрицы в виде произведения

$$
    A = Q R, 
$$

где $Q$ – матрица с **ортогональными столбцами** и $R$ – **верхнетреугольная**.  

- Размеры матриц: $Q$ – $n \times m$, $R$ – $m \times m$, если $n\geq m$.

- QR разложение определено для любой **прямоугольной матрицы**.

## Приложения QR разложения

Это разложение играет ключевую роль при решении многих задач, например:
- Получение ортогонального базиса в линейном пространстве
- Используется для препроцессинга при вычислении SVD
- QR алгоритм для вычисления собственных векторов и собственных значений ([один из 10 самых важных алгоритмов ХХ века](https://archive.siam.org/pdf/news/637.pdf)) основан на вычислении QR разложения
- Решение переопределённых систем линейных уравнений

## Существование QR разложения

**Теорема.**

Каждая матрица $n \times m$ может быть представлена в виде QR разложения. 


Существует несколько способов доказательства и вычисления:

- Теоретический: используя матрицу Грама и разложение Холецкого
- Геометрический: используя ортогонализацию Грама-Шмидта
- Практический: используя преобразования Гивенса/Хаусхолдера

### Доказательство с использованием разложения Холецкого

Если у нас есть разложение

$$A = QR,$$

тогда $A^* A = ( Q R)^* (QR)  = R^* (Q^* Q) R = R^* R$, матрица $A^* A$ называется **матрицей Грама**, и её элементы – скалярные произведения столбцов матрицы $A$.  

### Случай матрицы полного ранга

Пусть $A$ имеет **полный столбцовый ранг**. Тогда легко показать, что $A^* A$ положительно определена:

$$
   (A^* A y, y) = (Ay, Ay) = \Vert Ay \Vert^2_2  > 0, \quad y\not = 0.
$$

Поэтому, $A^* A = R^* R$ всегда существует.

Тогда матрица $A R^{-1}$ унитарна:  

$$
   (A R^{-1})^* (AR^{-1})= R^{-*} A^* A R^{-1} = R^{-*} R^* R R^{-1} = I.
$$


### Случай матрицы неполного ранга

- QR разложение по-прежнему существует.

- Для любой матрицы неполного ранга существует последовательность матриц полного ранга $A_k$ такая что $A_k \rightarrow A$ (почему?).

- Каждая $A_k$ может быть разложена $A_k = Q_k R_k$. 

- Множество унитарных матриц образует компакт, таким образом найдётся сходящаяся подпоследовательность $Q_{n_k} \rightarrow Q$ (почему?), и $R_{n_k} = Q^*_{n_k} A_{n_k} \rightarrow Q^* A = R$, которая верхнетреугольная.

## Устойчивость вычисления QR разложения с помощью разложения Холецкого 

Итак, простейший способ вычисления QR разложения следующий

$$A^* A = R^* R,$$

и

$$Q = A R^{-1}.$$

Это **плохая идея** с точки зрения устойчивости. Покажем, почему это так.

In [11]:
import numpy as np
n = 50
r = 8
a = [[1.0 / (i + j + 0.5) for i in range(r)] for j in range(n)]
a = np.array(a)
print(a.shape)
q, Rmat = np.linalg.qr(a)
e = np.eye(r)
print('Built-in QR orth', np.linalg.norm(np.dot(q.T, q) - e))
gram_matrix = a.T.dot(a)
Rmat1 = np.linalg.cholesky(gram_matrix)
q1 = np.dot(a, np.linalg.inv(Rmat1.T))
print('Via Cholesky:', np.linalg.norm(np.dot(q1.T, q1) - e))

(50, 8)
Built-in QR orth 1.924152493674034e-15
Via Cholesky: 0.04587925317113147


## Второй способ: ортогонализация Грама-Шмидта

- QR разложение – это способ записи процесса ортогонализации Грама-Шмидта
- Для данного набора векторов $a_1, \ldots, a_m$ мы хотим найти ортонормированный базис $q_1, \ldots, q_m$ такой чтобы каждый вектор $a_i$ представлялся как линейная комбинация векторов из базиса.  

**Метод Грама-Шмидта:**
1. $q_1 := a_1/\Vert a_1 \Vert$
2. $q_2 := a_2 - (a_2, q_1) q_1, \quad q_2 := q_2/\Vert q_2 \Vert$
3. $q_3 := a_3 - (a_3, q_1) q_1 - (a_3, q_2) q_2, \quad q_3 := q_3/\Vert q_3 \Vert$
4. И так далее  

Заметим, что преобразование из $A$ в $Q$ имеет треугольную структуру, поскольку мы вычитаем из $k$-го вектора только предыдущие. Это следует из того, что произведение треугольных матриц – это треугольная матрица.

## Модифицированный метод Грама-Шмидта

- Метод Грама-Шмидта может быть очень неустойчивым (то есть генерировать векторы, которая не являются ортогональными, особенно если $q_k$ малой нормы).  
Это называется **потеря ортогональности**.  

- Этот недостаток метода Грама-Шмидта исправляется с помощью **модифицированного метода Грама-Шмидта**. Вместо вычисления

$$q_k := a_k - (a_k, q_1) q_1 - \ldots - (a_k, q_{k-1}) q_{k-1}$$

мы будем вычислять это выражение шаг за шагом. Сначала присвоим $q_k := a_k$, затем последовательно ортогонализуем:

$$
   q_k := q_k - (q_k, q_1)q_1, \quad q_k := q_{k} - (q_k,q_2)q_2, \ldots
$$

- В точной арифметике, это одинаковые алгоритмы. В неточной арифметике они абсолютно разные!

- Заметим, что сложность по-прежнему $\mathcal{O}(nm^2)$ операций

In [29]:
import numpy as np

n = 50
r = 8
a = [[1.0 / (i + j + 0.5) for i in range(r)] for j in range(n)]
A = np.array(a)

def gramm_schmidt(A):
    Q = np.zeros_like(A)
    R = np.zeros((A.shape[1], A.shape[1]))
    n = A.shape[1]
    for i in range(A.shape[1]):
        Q[:, i] = A[:, i].copy()
        for j in range(i):
            R[j, i] = Q[:, i] @ Q[:, j]
        for j in range(i):
            Q[:, i] -= R[j, i] * Q[:, j]
        R[i, i] = np.linalg.norm(Q[:, i])
        Q[:, i] /= np.linalg.norm(Q[:, i])
    return Q, R

Q, R = gramm_schmidt(A)
print(np.linalg.norm(Q.T @ Q - np.eye(r)))
print(np.linalg.norm(A - Q @ R))

def modified_gramm_schmidt(A):
    Q = np.zeros_like(A)
    R = np.zeros((A.shape[1], A.shape[1]))
    n = A.shape[1]
    for i in range(A.shape[1]):
        Q[:, i] = A[:, i].copy()
        for j in range(i):
            R[j, i] = Q[:, i] @ Q[:, j]
            Q[:, i] -= Q[:, i] @ Q[:, j] * Q[:, j]
        R[i, i] = np.linalg.norm(Q[:, i])
        Q[:, i] /= np.linalg.norm(Q[:, i])
    return Q, R

Q, R = modified_gramm_schmidt(A)
print(np.linalg.norm(Q.T @ Q - np.eye(r)))
print(np.linalg.norm(A - Q @ R))

0.1728828047902182
1.5936813088446094e-16
1.2696433159012304e-09
1.658816568534105e-16


## QR разложение: почти практический способ

Если $A = QR$, тогда  

$$
R = Q^* A,
$$

и нам нужно найти такую ортогональную матрицу $Q$, которая преобразует данную матрицу $A$ в верхнетреугольную.  
Для простоты мы будем смотреть на матрицы $n \times n$ такие что

$$ Q^* A = \begin{bmatrix} * & * & *  \\ 0 & * & * \\ 0 & 0 & * \\ & 0_{(n-m) \times m} \end{bmatrix} $$

Будем приводить матрицу к такому виду столбец за столбцом.

Сначала найдём такую матрицу Хаусхолдера $H_1 = (I - 2 uu^{\top})$ что 

$$ H_1 A = \begin{bmatrix} * & * & * \\ 0 & * & * \\ 0 & * & * \\ 0 & * & * \end{bmatrix} $$

Затем

$$ H_2 H_1 A = \begin{bmatrix} * & * & * \\ 0 & * & * \\ 0 & 0 & * \\ 0 & 0 & * \end{bmatrix}, $$

где

$$ H_2 = \begin{bmatrix} 1 & 0 \\ 0 & H'_2, \end{bmatrix} $$

и $H'_2$ матрица Хаусхолдера $3 \times 3$.

И наконец, 

$$ H_3 H_2 H_1 A = \begin{bmatrix} * & * & * \\ 0 & * & * \\ 0 & 0 & * \\ 0 & 0 & 0 \end{bmatrix}, $$

где $H_3=\begin{bmatrix}I_2 & \\ & {\widetilde H}_3 \end{bmatrix}$ такая что

$$ 
{\widetilde H}_3 \begin{bmatrix} \boldsymbol{\times}\\ \boldsymbol{\times} \\ \boldsymbol{\times}  \end{bmatrix} = 
\begin{bmatrix} \times \\ 0 \\ 0  \end{bmatrix}.
$$

Попробуйте самостоятельно реализовать такой алгоритм, это просто!

### Получение QR разложения

Так как 

$$ H_3H_2H_1A = HA = R,$$

где $H$ – унитарная матрица, то

$$ A = H^*R. $$

Таким образом $Q = H^*$.

## QR разложение: практический способ

- Поскольку мы работаем с плотной матрицей, то на практике нам нужен алгоритм, оперирующий блоками (почему?).  

- Вместо использования преобразования Хаусхолдера, мы будем использовать **блочное преобразование Хаусхолдера** вида 

$$H = (I - 2UU^*), $$

где $U^* U = I$.

- Это позволяет использовать BLAS-3 операции

## QR разложение: практический способ - 2

Аналогично преобразованию Хаусхолдера для вычисления QR разложения можно использовать преобразование Гивенса

$$\begin{bmatrix} \times & \times & \times \\ \bf{*} & \times & \times \\ \bf{*} & \times & \times \end{bmatrix} \to \begin{bmatrix} * & \times & \times \\ * & \times & \times \\ 0 & \times & \times \end{bmatrix} \to \begin{bmatrix} \times & \times & \times \\ 0 & * & \times \\ 0 & * & \times \end{bmatrix} \to \begin{bmatrix} \times & \times & \times \\ 0 & \times & \times \\ 0 & 0 & \times \end{bmatrix} $$

## Преобразование Гивенса vs. преобразование Хаусхолдера

- Матрицы Хаусхолдера полезны для плотных матриц (сложность примерно в два раза меньше), в которых необходимо занулить большое число элементов за одно отражение.
- Вращения Гивенса больше подходят для работы с разреженными матрицами или параллельными вычислениями, поскольку они зануляют только один элемент каждым действием.

## QR разложение: итоги

- Существует для любой матрицы
- Геометрически означает ортогонализацию векторов
- Эффективные алгоритмы основаны на BLAS-3 операциях